In [2]:
#pip install folium
#pip install geographiclib
import urllib.request, json
import pandas as pd
from pandas.io.json import json_normalize
import math
#Google MapsDdirections API endpoint
endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
api_key = '<yourapikey>'

#Asks the user to input Where they are and where they want to go.
origin = input('Where are you now?: ').replace(' ','+')
destination = input('Where do you want to go?: ').replace(' ','+')

#Building the URL for the request
nav_request = 'origin={}&destination={}&key={}'.format(origin,destination,api_key)
request = endpoint + nav_request

#Sends the request and reads the response.
response = urllib.request.urlopen(request).read()

#Loads response as JSON
directions = json.loads(response)
#print(directions)

routes = directions['routes']
routes

# To obtain latitude and longitude
source_latitude = routes[0]['legs'][0]['start_location']['lat']
source_longitude = routes[0]['legs'][0]['start_location']['lng']
dest_latitude = routes[0]['legs'][0]['end_location']['lat']
dest_longitude = routes[0]['legs'][0]['end_location']['lng']

print(f"\n\n\nyour current  co-ordinates are \n\t Latitude  : {source_latitude} \n\t longitude : {source_longitude}")
print(f"\n\n\nyour destination co-ordinates are \n\t Latitude  : {dest_latitude} \n\t longitude : {dest_longitude}")

print("\n\n\n\ncalculating the midpoint of the journey \n. \n. \n. \n. \n. \n. \n.")

import sys
sys.path.append("/usr/local/lib/python/site-packages")
from geographiclib.geodesic import Geodesic


def midpoint(x1, y1, x2, y2):
    
    # Define the path from 1 to 2
    l = Geodesic.WGS84.InverseLine(x1, y1, x2, y2)

    # Compute the midpoint
    m = l.Position(0.5 * l.s13)
    #print(m['lat2'],m['lon2'])
    return [round(m['lat2'], 7), round(m['lon2'], 7)]

mid_lat, mid_lon = midpoint(source_latitude,source_longitude,dest_latitude,dest_longitude)
print(f"your midpoint co-ordinates are \n\t Latitude  : {mid_lat} \n\t longitude : {mid_lon}")
import requests
import json

def get_hotels(lat,lon):

        # Define a business ID
    business_id = 'eNl0VYhOAlL9Pi9rSZG9hw'
    unix_time = 1546047836

    # Define my API Key, My Endpoint, and My Header
    API_KEY = 'X4GONbIPUjiWuSAh8Vt2pAIhg5JljLLT-_FFwe2jummKx_wLUEI2cFPM4CKDI-qU1jeTcPV6p3YSFg_RY9PYq2kMgWuEkmDy88xHk1PuK9AE9CbzTSubPdO1JjXaXHYx'
    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    HEADERS = {'Authorization': 'bearer %s' % API_KEY}

    # Define my parameters of the search
    # BUSINESS SEARCH PARAMETERS - EXAMPLE
    PARAMETERS = {'term': 'restaurants',
                  'latitude': lat,
                  'longitude' : lon,
                  'radius': 25000,
                  'categories' : "food, All",
                  'limit': 50,
                  'offset': 50,
                  'sort_by' : 'distance'
                 }

    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                            params = PARAMETERS,
                            headers = HEADERS)

    # Convert the JSON String
    business_data = response.json()
    business_data

    import pandas as pd
    from pandas.io.json import json_normalize

    a = json_normalize(business_data['businesses'])
    columns = ['name','location.display_address','rating','distance','is_closed','coordinates.latitude','coordinates.longitude']
    result = a.reindex(columns=columns)
    return result

source_df = get_hotels(source_latitude, source_longitude)
dest_df = get_hotels(dest_latitude, dest_longitude)
mid_df = get_hotels(mid_lat, mid_lon)

frames = [source_df,dest_df,mid_df]
final_dataframe = pd.concat(frames).reset_index(drop=True).sort_values(by=['rating', 'distance'], ascending=[False, True])

def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 

color_dictionary ={5 : 'green', 4.5:'green', 4:'green', 3.5:'orange', 3:'orange',2.5:'orange',2:'red',1.5:'red', 1:'red', 0.5:'red', 0.0:'red'} 
final_dataframe['color'] = final_dataframe['rating'].apply(set_value, args =(color_dictionary, )) 

icon_dictionary = {'green':'thumbs-up', 'orange':'thumbs-up', 'red':'thumbs-down'}
final_dataframe['icon'] = final_dataframe['color'].apply(set_value, args =(icon_dictionary, ))

final_dataframe['popup'] = final_dataframe['name'].map(str) + '                    ' + final_dataframe['location.display_address'].map(str) + '                        ' + round(final_dataframe['distance']).map(str) + ' metres'

##### creating maps 
from collections import namedtuple
import numpy as np
p1 = [source_latitude, source_longitude]
p2 = [dest_latitude, dest_longitude]

def get_arrows(locations, color='blue', size=8, n_arrows=4):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation).add_to(m))
    return arrows

def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

m = folium.Map(
    location=[mid_lat, mid_lon],
    zoom_start=4,
    tiles='Stamen Terrain'
)

import folium
def get_map(dataframe):

    for index, row in dataframe.iterrows():

        folium.Marker([row['coordinates.latitude'],row['coordinates.longitude']],
                      popup=row['popup'],
                      icon=folium.Icon(icon=row['icon'],color=row['color'])
                       ).add_to(m)
        folium.Marker(location=[mid_lat, mid_lon],
                      popup='middle_point',
                      icon=folium.Icon(color='red', icon='centrecode')
                      ).add_to(m)
        folium.PolyLine(locations=[p1, p2], color='blue').add_to(m)
        arrows = get_arrows(locations=[p1, p2], n_arrows=4)
    for arrow in arrows:
        arrow.add_to(m)

    m.save('hotel_map.html')
    return m 
get_map(final_dataframe)


Where are you now?: buffalo
Where do you want to go?: dallas



your current  co-ordinates are 
	 Latitude  : 42.8864931 
	 longitude : -78.8787931



your destination co-ordinates are 
	 Latitude  : 32.7759428 
	 longitude : -96.7967532




calculating the midpoint of the journey 
. 
. 
. 
. 
. 
. 
.
your midpoint co-ordinates are 
	 Latitude  : 38.1744069 
	 longitude : -88.4557014


In [161]:
final_dataframe

,name,location.display_address,rating,distance,is_closed,coordinates.latitude,coordinates.longitude,color,icon,popup
0,Perks Cafe at Horsefeathers,"[346 Connecticut St, Buffalo, NY 14213]",5.0,2287.007869,0.0,42.905850,-78.888290,green,thumbs-up,Perks Cafe at Horsefeathers ...
69,Just Encased Food Truck,"[1408 North Riverfront Blvd, Dallas, TX 75207]",5.0,2710.569535,0.0,32.790410,-96.820080,green,thumbs-up,Just Encased Food Truck ['1...
71,The Bodega at Hi Line,"[1400 Hi Line Dr, Ste 120, Dallas, TX 75207]",5.0,2807.586674,0.0,32.795330,-96.816050,green,thumbs-up,The Bodega at Hi Line ['140...
80,Mz Deli & Food Mart,"[4420 Worth St, Dallas, TX 75246]",5.0,3152.540893,0.0,32.794579,-96.771339,green,thumbs-up,Mz Deli & Food Mart ['4420 ...
14,Pattaya Street Food,"[25 Grant St, West Side Bazaar, Buffalo, NY 14...",5.0,3289.040591,0.0,42.914856,-78.890389,green,thumbs-up,Pattaya Street Food ['25 Gr...
22,Kathrene’s Cuisine,"[999 Broadway St., The Broadway Market, Buffal...",5.0,3431.184544,0.0,42.893166,-78.837673,green,thumbs-up,Kathrene’s Cuisine ['999 Br...
94,Soulgood,"[Dallas, TX 75215]",5.0,3691.956492,0.0,32.767811,-96.768120,green,thumbs-up,"Soulgood ['Dallas, TX 75215..."
40,La Verdad Cafe,"[1132 E Lovejoy St, Buffalo, NY 14206]",5.0,5910.400024,0.0,42.890030,-78.806400,green,thumbs-up,La Verdad Cafe ['1132 E Lov...
49,Hearty House,"[2181 Clinton St, West Seneca, NY 14206]",5.0,6958.651690,0.0,42.871420,-78.795910,green,thumbs-up,Hearty House ['2181 Clinton...
54,Café Victoria,"[2422 Victory Park Ln, Ste 100, Dallas, TX 75219]",4.5,1792.112643,0.0,32.788145,-96.809277,green,thumbs-up,Café Victoria ['2422 Victor...
